<a href="https://colab.research.google.com/github/coordanalisis-sedatu/datavivi/blob/main/siete_elementos_viv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cálculo de indicadores de los 7 elementos de la vivienda adecuada

Este notebook tiene como objetivo calcular indicadores que permitan caracterizar los 7 elementos de la vivienda adecuada. Esto, de acuerdo a un trabajo previo realizado en la SEDATU.

Las fuentes de información que se utilizarán son:
- Encuesta Nacional de Ingresos y Gastos de los Hogares (ENIGH) 2022
- Encuesta Nacional de Vivienda (ENVI) 2020
- Registro de vivienda del Sistema Nacional de Información e Indicadores de Vivienda (SNIIV) 2022

In [ ]:
##Borrar datos del entorno
rm(list=ls())

In [ ]:
##Crear folders de almacenamiento
dir.create("microdatos", showWarnings = F)

In [ ]:
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse,srvyr,googledrive,janitor,survey,
               sf,httr,jsonlite)

In [ ]:
# Función para descargar y descomprimir un archivo con timeout
download_and_unzip <- function(url, destination="microdatos") {
  tryCatch({
    # Descargar el archivo con timeout
    download.file(url, destfile = paste0(destination, "/", basename(url)), mode = "wb", timeout = 10000)

    # Descomprimir el archivo
    unzip(paste0(destination, "/", basename(url)), exdir = destination)

    # Eliminar el archivo comprimido después de descomprimirlo
    unlink(paste0(destination, "/", basename(url)))

    message(paste("Descarga y descompresión exitosas para", basename(url)))
  }, error = function(e) {
    warning(paste("Error al descargar o descomprimir", basename(url), ":", conditionMessage(e)))
  })
}


# Cálculos con datos de la ENIGH 2022

In [ ]:
download_and_unzip("https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/microdatos/enigh2022_ns_viviendas_csv.zip")
download_and_unzip("https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/microdatos/enigh2022_ns_hogares_csv.zip")


Descarga y descompresión exitosas para enigh2022_ns_viviendas_csv.zip

Descarga y descompresión exitosas para enigh2022_ns_hogares_csv.zip



In [ ]:
# Carga y limpia el conjunto de datos de viviendas para el año 2022
enigh22 <- read.csv("microdatos/viviendas.csv") %>%
  janitor::clean_names() %>%  #Poner las variables en minusculas y quitar las variables
  mutate(
    mat_pared = as.numeric(mat_pared),
    mat_pisos = as.numeric(mat_pisos),
    mat_techos = as.numeric(mat_techos),
#Crear variable de rezago habitacional
    rezago = case_when(
      (tot_resid / num_cuarto) > 2.5 |   # Hacinamiento
        mat_pared %in% c(1, 2, 3, 4, 5, 6) |
        mat_pisos %in% c(1) |
        mat_techos %in% c(1, 2, 3, 4, 6, 7, 9) |
        excusado == 2 ~ "En rezago",
      TRUE ~ "Fuera de rezago"
    )
  )

Warning message:
"There was 1 warning in `mutate()`.
ℹ In argument: `mat_pisos = as.numeric(mat_pisos)`.
Caused by warning:
! NAs introduced by coercion"


In [ ]:
#Definir diseño muestral para ambos años
dm<-enigh22%>%
  #Diseño muestral
  as_survey(weights=factor, strata=est_dis, ids=upm)

###  Habitabilidad


In [ ]:
print("Viviendas en rezago habitacional")
dm%>%
  group_by(rezago)%>%
  summarise(
    porcentaje=survey_prop(vartype="cv"))%>%
    mutate(porcentaje=round(porcentaje*100,1))%>%
filter(rezago %in% c("En rezago"))

[1] "Viviendas en rezago habitacional"


rezago,porcentaje,porcentaje_cv
<chr>,<dbl>,<dbl>
En rezago,24.3,0.01054548


### Seguridad de la tenencia

In [ ]:
print("Viviendas propias y propias en proceso de pago sin escrituras")

dm%>%
#Filtrar propias y propias se están pagando
filter(tenencia == 3 | tenencia == 4)%>%
mutate(escrituras=case_when(
  escrituras==1~"A nombre del dueño",
  escrituras==2~"A nombre de otra persona",
    escrituras==3~"No tiene escrituras",
    escrituras==4~"No sabe si tiene escrituras",
))%>%
group_by(escrituras)%>%
  summarise(
    porcentaje=survey_prop(vartype="cv"))%>%
    mutate(porcentaje=round(porcentaje*100,1))%>%
    filter(escrituras %in% c("No tiene escrituras"))

[1] "Viviendas propias y propias en proceso de pago sin escrituras"


escrituras,porcentaje,porcentaje_cv
<chr>,<dbl>,<dbl>
No tiene escrituras,24.3,0.01473525


### Disponibilidad de servicios, materiales, instalaciones e infraestructura

Para evitar el cálculo del indicador, se hará un join con la tabla de pobreza del CONEVAL

In [ ]:
url_coneval<-"https://www.coneval.org.mx/Medicion/MP/Documents/MMP_2022/Programas_calculo/R_MMP_2022.zip"
download_and_unzip(url_coneval)

Warning message in value[[3L]](cond):
"Error al descargar o descomprimir R_MMP_2022.zip : invalid multibyte string at '<a0>lcu'"


In [ ]:
pobreza<-read.csv("microdatos/Base final/pobreza_22.csv")%>%
select(folioviv, foliohog,numren,ic_sbv)

str(pobreza)

'data.frame':	309534 obs. of  4 variables:
 $ folioviv: num  1e+08 1e+08 1e+08 1e+08 1e+08 ...
 $ foliohog: int  1 1 1 1 1 1 1 1 1 1 ...
 $ numren  : int  1 2 3 1 2 1 2 3 1 2 ...
 $ ic_sbv  : int  0 0 0 0 0 0 0 0 0 0 ...


In [ ]:
vivsb<-enigh22%>%
  left_join(
    pobreza %>%
      group_by(folioviv) %>%
      filter(row_number() == 1) %>%
      select(folioviv,ic_sbv),
    by = "folioviv"
  )

In [ ]:
#Definir diseño muestral
dm_viv<-vivsb%>%
  as_survey(weights=factor, strata=est_dis, ids=upm)

In [ ]:
print("Viviendas con falta de acceso a servicios básicos")
dm_viv%>%
group_by(ic_sbv)%>%
    summarise(porcentaje=survey_prop(vartype="cv"))%>%
    mutate(porcentaje=round(porcentaje*100,1))%>%
    filter(ic_sbv %in% c(1))

[1] "Viviendas con falta de acceso a servicios básicos"


ic_sbv,porcentaje,porcentaje_cv
<int>,<dbl>,<dbl>
1,16.2,0.01616862


### Asequibilidad

In [ ]:
pago_vivienda<-enigh22%>%
#Se considera únicamente información de aquellas viviendas propias que se están pagando
filter(tenencia %in% c(3))%>%
select(folioviv, tenencia, pago_viv)
head(pago_vivienda)

,folioviv,tenencia,pago_viv
,<dbl>,<int>,<int>
1,100224706,3,2700
2,100336806,3,2200
3,100385904,3,20000
4,100386703,3,9000
5,100386704,3,14000
6,100404202,3,12000


In [ ]:
#Uilizar la tabla de concentrado hogar
download_and_unzip("https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/microdatos/enigh2022_ns_concentradohogar_csv.zip")

Descarga y descompresión exitosas para enigh2022_ns_concentradohogar_csv.zip



In [ ]:
concentrado<-read.csv("microdatos/concentradohogar.csv")%>%
  janitor::clean_names()%>%
#Ingreso trimestral a mensual
mutate(
  ing_cor_mensual=ing_cor/3
)%>%
#Pegar datos de pago de vivienda
left_join(
  pago_vivienda,
  by="folioviv"
)%>%
#Cálculo de porcentaje de ingreso destinado al pago de vivienda
mutate(
  porcentaje_pago=(pago_viv/ing_cor_mensual)*100
)%>%
#Hacer variable categórica de acuerdo a la proporción de ingreso destinado al pago de vivienda
mutate(
  asequibilidad=case_when(
    porcentaje_pago<=30~"Hasta el 30%",
    porcentaje_pago>30~"Más del 30%"
  )
)

In [ ]:
#Definir diseño muestral
dm_concentrado<-concentrado%>%
  as_survey(weights=factor, strata=est_dis, ids=upm)

In [ ]:
#Calcular porcentaje de hogares con más del 30% de ingreso destinado al pago de vivienda
print("Porcentaje de hogares que destina hasta el 30% de su ingreso al pago de vivienda")
dm_concentrado%>%
  group_by(asequibilidad)%>%
  summarise(
    porcentaje=survey_prop(vartype="cv"))%>%
    mutate(porcentaje=round(porcentaje*100,1))%>%
    filter(asequibilidad %in% c("Hasta el 30%"))

[1] "Porcentaje de hogares que destina hasta el 30% de su ingreso al pago de vivienda"


asequibilidad,porcentaje,porcentaje_cv
<chr>,<dbl>,<dbl>
Hasta el 30%,7.4,0.02343417



# Cálculos con datos de la ENVI 2020

In [ ]:
download_and_unzip("https://www.inegi.org.mx/contenidos/programas/envi/2020/microdatos/envi_2020_base_de_datos_csv.zip")

Descarga y descompresión exitosas para envi_2020_base_de_datos_csv.zip



### Adecuación cultural

In [ ]:
#Carga de los datos

envi2020<-read.csv("microdatos/Bases de datos/TVIVIENDA.csv")%>%
  janitor::clean_names()

In [ ]:
#Declaración del diseño muestral
options(survey.lonely.psu="adjust")
dm_envi<-envi2020%>%
  as_survey(weights= factor, strata=est_dis, ids=upm_dis)

In [ ]:
print("Satisfacción con la vivienda de acuerdo a adecuación cultural")
dm_envi%>%
  group_by(p6_6)%>%
mutate(p6_6=case_when(
    p6_6==1~"Mucho",
    p6_6==2~"Algo",
    p6_6==3~"Poco",
    p6_6==4~"Nada",
    p6_6==9~"No especificado"))%>%
  summarise(porcentaje=survey_prop(vartype="cv"))%>%
  mutate(porcentaje=round(porcentaje*100,1))%>%
    filter(p6_6 %in% c("Mucho"))

[1] "Satisfacción con la vivienda de acuerdo a adecuación cultural"


p6_6,porcentaje,porcentaje_cv
<chr>,<dbl>,<dbl>
Mucho,51.2,0.006897074


### Accesibilidad

In [ ]:
print("Viviendas con necesidad de adecuaciones para personas con discapacidad")
dm_envi%>%
  mutate(discap=
case_when(
  p6_7_1==1 |
p6_7_2==1 |
p6_7_3==1 |
p6_7_4==1  ~"Con necesidad de adecuación",
TRUE~"Sin necesidad de adecuación"
  )
)%>%
group_by(discap)%>%
  summarise(
    porcentaje=survey_prop(vartype="cv"))%>%
    mutate(porcentaje=round(porcentaje*100,1))%>%
    filter(discap %in% c("Con necesidad de adecuación"))

[1] "Viviendas con necesidad de adecuaciones para personas con discapacidad"


discap,porcentaje,porcentaje_cv
<chr>,<dbl>,<dbl>
Con necesidad de adecuación,16.9,0.0142452


# Cálculos con datos del SNIIV
### Ubicación

Para este indicador se utilizará la información del registro de vivienda que se publica en el Sistema Nacional de Información e Indicadores de Vivienda (SNIIV) de la Secretaría de Desarrollo Agrario, Territorial y Urbano (SEDATU).

In [ ]:
print("Viviendas fuera de contorno")

url<-"https://sniiv.sedatu.gob.mx/api/CuboAPI/GetRegistro/2022,2022/0/0/pcu,anio"

ubicacion<-GET(url)
ubicacion<-content(ubicacion,as="text")
ubicacion<-fromJSON(ubicacion)%>%
#Calcular porcentajes
mutate(

#Pcu numérico
viviendas=as.numeric(viviendas),
  porcentaje=round((viviendas/sum(viviendas))*100,1)
)%>%
filter(pcu %in% c("FC"))

ubicacion

[1] "Viviendas fuera de contorno"


año,pcu,viviendas,porcentaje
<int>,<chr>,<dbl>,<dbl>
2022,FC,21480,15
